### aim: 

### date: 

In [8]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [9]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [10]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
%matplotlib inline


In [11]:
def get_limiter(run = 'TOM12_TJ_LC00', year = 1920):

    tdir = f'/gpfs/data/greenocean/software/runs/{run}/'

    tfi = f'ORCA2_1m_{year}0101_{year}1231_diad_T.nc'

    w = xr.open_dataset(f'{tdir}/{tfi}')
    print(f'{run} {year}')
    
    tmesh = xr.open_dataset('/gpfs/data/greenocean/software/resources/regrid/mesh_mask3_6.nc')
    tm = tmesh.tmask.values
    ## make meshmask of same size as data 
    tm_broad = np.zeros([12,31, 149, 182])
    for i in range(0,12):
        tm_broad[i,:,:,:] = tm[0,:,:,:]
    #lim3fe_dia', 'lim3fe_mix', 'lim3fe_coc', 'lim3fe_pic', 'lim3fe_pha', 'lim3fe_fix', 
    pfts = ['dia','mix','coc','pic','pha','fix']

    for pft in pfts:

        A = f'lim3fe_{pft}'
        B = f'lim4po4_{pft}'
        if pft == 'dia':
            C = f'lim5si_{pft}'
        D = f'lim6din_{pft}'
        E = f'LV_{pft}'
        if pft == 'dia':
            stacked = xr.concat([w[A], w[B], w[C], w[D]], dim='variable')
        else:
            stacked = xr.concat([w[A], w[B], w[D]], dim='variable')

        stacked = stacked.where(stacked != 0, np.nan)
        stacked = stacked.fillna(np.inf)

        # Find the minimum values across the new 'variable' dimension
        w[E] = stacked.min(dim='variable', skipna = True)

        # Find the index of the minimum values (this will give 0, 1, 2, 3
        # which we later transform to 10 20 30 40)
        min_indices = stacked.argmin(dim='variable', skipna = True)
        q2 = np.copy(min_indices).astype(float)
        q2[np.where(tm_broad == 0)] = np.nan
        # Add 1 to match the [1, 2, 3, 4] numbering scheme
        q3 = (q2+1)*10 

        if pft == 'dia': #silica lim is 5
            q3[q3 == 10] = 3
            q3[q3 == 20] = 4
            q3[q3 == 30] = 5
            q3[q3 == 40] = 6
        else:
            q3[q3 == 10] = 3
            q3[q3 == 20] = 4
            q3[q3 == 30] = 6
            
        q3[np.where(tm_broad == 0)] = np.nan
        w[f'LN_{pft}'] = w[f'LV_{pft}']
        w[f'LN_{pft}'].data = q3


    w_sel = w[['LV_dia', 'LN_dia', 'LV_mix', 'LN_mix', 'LV_coc', 'LN_coc', 'LV_pic', 'LN_pic', 'LV_pha', 'LN_pha', 'LV_fix', 'LN_fix']]
    
    try:
        w_sel.to_netcdf(f'{tdir}/ORCA2_1m_{year}0101_{year}1231_LoP_T.nc')
        print(f'saved {run} {year}:')
        print(f'{tdir}/ORCA2_1m_{year}0101_{year}1231_LoP_T.nc')
        print()
    except:
        print(f'failed 2 save {run} {year}')
        print()

In [12]:
ex = True
if ex:
    for year in range(1955,2024):

        get_limiter('TOM12_TJ_LA02', year)
        get_limiter('TOM12_TJ_LC02', year)

TOM12_TJ_LA02 1955
saved TOM12_TJ_LA02 1955:
/gpfs/data/greenocean/software/runs/TOM12_TJ_LA02//ORCA2_1m_19550101_19551231_LoP_T.nc

TOM12_TJ_LC02 1955
saved TOM12_TJ_LC02 1955:
/gpfs/data/greenocean/software/runs/TOM12_TJ_LC02//ORCA2_1m_19550101_19551231_LoP_T.nc

TOM12_TJ_LA02 1956
saved TOM12_TJ_LA02 1956:
/gpfs/data/greenocean/software/runs/TOM12_TJ_LA02//ORCA2_1m_19560101_19561231_LoP_T.nc

TOM12_TJ_LC02 1956
saved TOM12_TJ_LC02 1956:
/gpfs/data/greenocean/software/runs/TOM12_TJ_LC02//ORCA2_1m_19560101_19561231_LoP_T.nc

TOM12_TJ_LA02 1957
saved TOM12_TJ_LA02 1957:
/gpfs/data/greenocean/software/runs/TOM12_TJ_LA02//ORCA2_1m_19570101_19571231_LoP_T.nc

TOM12_TJ_LC02 1957
saved TOM12_TJ_LC02 1957:
/gpfs/data/greenocean/software/runs/TOM12_TJ_LC02//ORCA2_1m_19570101_19571231_LoP_T.nc

TOM12_TJ_LA02 1958
saved TOM12_TJ_LA02 1958:
/gpfs/data/greenocean/software/runs/TOM12_TJ_LA02//ORCA2_1m_19580101_19581231_LoP_T.nc

TOM12_TJ_LC02 1958
saved TOM12_TJ_LC02 1958:
/gpfs/data/greenocean/so

FileNotFoundError: [Errno 2] No such file or directory: b'/gpfs/data/greenocean/software/runs/TOM12_TJ_LA02/ORCA2_1m_20040101_20041231_diad_T.nc'

In [ ]:
# run = 'TOM12_TJ_LA02'
# year = 1985
# tdir = f'/gpfs/data/greenocean/software/runs/{run}/'

# tfi = f'ORCA2_1m_{year}0101_{year}1231_diad_T.nc'

# w = xr.open_dataset(f'{tdir}{tfi}')

# print(list(w.data_vars.keys()))